In [189]:
import pandas as pd
import re

In [190]:
ml = pd.read_csv("casas_mercadoLibre.csv")

In [191]:
ml["property_type"] = "Casa"
ml = ml.drop_duplicates()
ml = ml.dropna()

In [192]:
# Elimino una fila cuyo valor no sirve para el análisis ya que es una 
# propiedad pensada para uso comercial y no una casa
ml = ml.drop(index=2).reset_index(drop="first")

### Econtrar en la descripción los valores de la columna parking

In [193]:
# Se tienen 9 filas que marcan como 0 el valor de parking
len(ml.query("parking==0"))

9

In [194]:
# Con esta función localizo el texto donde especifica los lugares de estacionamiento en la descripción
def econtrar_textos(texto):
    regex = r"(?i)(.{0,4}estacionamientos?.{0,20}|.{0,4}cocheras?.{0,20}|.{0,4}garages?.{0,20}|.{0,4}parkings?.{0,20})"
    # regex = r"(?i)(.{0,10})(estacionamientos?|cocheras?|garages?|parkings?)(.{0,30})"
    expresion = re.findall(regex, texto)
    expresion = "".join(expresion)
    if len(expresion) > 0:
        valor_de_celda = expresion
    else:
        valor_de_celda = "cero"
    return valor_de_celda

In [195]:
# Función para encontrar en la descripción el número de lugares de estacionamiento
def asignar_valor_estacionamiento(texto):
    # Buscar dígitos numéricos en el texto
    if texto != "cero":    
        numeros_encontrados = re.findall(r'\d+', texto)

        if numeros_encontrados:
            # Tomar el primer número encontrado y convertirlo a entero
            return int(numeros_encontrados[0])
        elif "para dos" in texto.lower():
            return 2
        elif "para tres" in texto.lower():
            return 3
        else:
            return 1

In [196]:
# Aplicar las funciones para buscar los valores de parking
ml["estacionamiento"] = ml["description"].apply(econtrar_textos)
ml["estacionamiento"] = ml["estacionamiento"].apply(asignar_valor_estacionamiento)

In [197]:
# Reemplazar los valores en las filas de la columna parking donde marca cero
ml.loc[7:12, "parking"] = ml.loc[7:12, "estacionamiento"]

In [198]:
# Se logró reducir de 9 a 3 el número de filas con valor 0 
# en la columna parking
len(ml.query("parking==0"))

3